<a id="intro"></a>
# Reminder About Key Distribution

**What is key distribution?** Secure communication relies on the ability of the sender (Alice) to encrypt the message in a way that the receiver (Bob) can decrypt it but not an eavesdropper. This security is often accomplished with the use of a **key**, which is a piece of information known only to the sender and receiver and enables them to decrypt and encrypt the message. If a key can be securely distributed between the sender and receiver, the encrypted message can be securely sent over a public channel, since without the key, the probability of successfully decrypting the message is tiny.

Practically, **a key is just a bitstring** - a sequence of 1s and 0s, that is uniquely known only to Alice and Bob, the two communicating parties.

Therefore, the problem of secure communication boils down to secure key distribution. QKD is unique because security against eavesdropping is guaranteed by the laws of quantum mechanics, as opposed to the computational complexity of certain functions, which is what is used in classical key distribution. 

In this notebook, we will develop qiskit code to implement Quantum Key Distribution (QKD). We will look at the same BB84 protocol that we just discussed in the Workshop. 

## The BB84 protocol

The goal of the BB84 protocol is to create and securely share a key, which is just a series of 0s and 1s, between the sender (Alice) and the receiver (Bob). The BB84 protocol achieves this in 3 phases:

### Phase 1 - Sending
Alice radomly generates a *bitstring* and a list of *bases*. Some of the bits of the bitstring will make it onto the key. The *bases* can be either X or Z, selected randomly. Next, Alice **encodes** each bit of the bistring into a qubit using the corresponding basis in the list of bases, using the following decision scheme: 

| bit in bitstring | basis | State Alice encodes |
|:----------------:|:--------------------------:|:---------------:|
| 0 | Z | $$|0\rangle$$ |
| 1 | Z | $$|1\rangle$$ |
| 0 | X | $$|+\rangle$$ |
| 1 | X | $$|-\rangle$$ |

Alice sends each of these encoded qubits to Bob. So, Alice can send Bob one of **four** possible states - $|0\rangle, |1\rangle, |+\rangle$, and $|-\rangle$.


### Phase 2 - Receiving
Bob receives this qubit, and needs to measure them to find out their state. Bob generates a random list of bases, either X or Z, and measures each of the qubits he receives in the corresponding basis to generate his own bitstring. If Bob measures the qubit in the same basis that Alice had encoded it in, he will get the same bit as a result of that measurement as Alice had encoded. If Bob measures the qubit in a different basis, he is not guaranteed to get the same bit as Alice. 

### Phase 3 - Comparing
Therefore, after Bob has measured all qubits that Alice sent him, Alice and Bob publically declare their list of bases, and drop the bits from their respective bitstrings for which their **bases do not match**. The remaining bits, for which their bases match, are **guaranteed to match** as long as no one was eavesdropping when Alice sent Bob her qubits. Therefore, to find out if someone was eavesdropping, Alice and Bob can compare the first few bits of their respective keys and see if they match.

The figure below summarizes the BB84 protocol:

In [6]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from copy import deepcopy

from random import getrandbits

print("Libraries imported successfully!")

Libraries imported successfully!


## Coding cheat sheet:
### Defining a quantum circuit: 
```python
qc = QuantumCircuit(1,1) # Define a 1 qubit, 1 classical bit quantum circuit

qc.x(0) #Add an X gate
qc.h(0) #Add an H gate
qc.z(0) #Add a Z gate
qc.y(0) #Add a Y gate

qc.draw() #Draw the circuit
```

### Using the qasm simulator:
First we have to add measurement gates:

``` python
qc.measure_all() #adds measurements
```
Next
``` python
qsim = Aer.get_backend('qasm_simulator') # Change statevector to qasm
job = execute(qc, backend=qsim, shots=1000) # add shots - tell it how many times to run, more shots = lower noise
result = job.result()
```

Lastly, we can visualise the output in histogram form:
``` python
counts = result.get_counts(qc)
plot_histogram(counts)
```

### Using a real quantum computer:

First we need to find the least busy backend:
```python
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 
                                       and not x.configuration().simulator 
                                       and x.status().operational==True))
print("least busy backend: ", backend)
```

Next we can send the job to be run"
``` python
job = execute(qc, backend=backend, shots=100)
result = job.result()
```

Lastly, we can again plot the results in the same way:

``` python
counts = result.get_counts(qc)
plot_histogram(counts)
```

<a id="protocol"></a>
# Steps in the QKD protocol

We will now implement the steps in the BB84 protocol one-by-one.

<a id="step1"></a>
## Step 1: Alice randomly picks bits

Alice needs to randomly select bits that will later get encoded into qubits to send to Bob. Some of these bits will make it onto Alice's (and Bob's) final key.

Write a function that takes the length of the key as an input parameter and generates a list called `alice_bits` of randomly selected bits (0s and 1s) using the `getrandbits` function. Store the randomly generated bits as strings in the list.

In [7]:
# Block 1 - Write a function to generate alice_bits
def generate_alice_bits(length):
    
    #This stores alice's bits
    alice_bits = []
    
    # For the length 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # The "1" in the function argument is the number of bits to be generated
        alice_bits.append(str(getrandbits(1)))
    
    # return the string of bits
    return alice_bits


## Step 2: Alice randomly picks bases

Alice needs to randomly select bases to encode her bits from the previous step. She will pick randomly from two bases - the Z basis and the X basis.

Write a function that takes the length of the key as an input parameter and generates a list called `alice_bases` of randomly selected bases ("X" and "Z"). Store the randomly generated bases as strings in the list.

In [8]:
# Block 2 - Write a function to generate alice_bases
def generate_alice_bases(length):
    
    #This stores alice's bases
    alice_bases = []
    
    # For the length 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # The "1" in the function argument is the number of bits to be generated
        basis = getrandbits(1)
        if basis == 0:
            alice_bases.append("Z")
        else:
            alice_bases.append("X")
        
    
    # return the string of bases
    return alice_bases

# Create `alice_bits` and `alice_bases`

Let's create our `alice_bits` and `alice_bases`. Call the functions you defined above to generate the two lists.

We can see what they look like by printing the first 10 elements, this should look like a string and a list of random '1's or '0's (for `alice_bits`) and a string of random 'Z's and 'X's (for `alice_bases`).

In [9]:
# Block 3 - Use your functions to create alice's lists of bits and bases

N = 500 # This is number of bits and bases Alice will generate
alice_bits = generate_alice_bits(N)
alice_bases = generate_alice_bases(N)

# Preview the first 10 elements of each:
print("alice's first 10 bits: ", alice_bits[:10])
print("alice's first 10 bases: ", alice_bases[:10])

alice's first 10 bits:  ['1', '0', '1', '0', '0', '1', '1', '1', '0', '0']
alice's first 10 bases:  ['X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X']


## Step 3: Alice encodes her bits using her bases

Alice now uses her random list of bits and bases to generate alist of qubits to send to Bob:
In this Challenge we are going to represent each qubit as an individual `QuantumCircuit` object.

The table below summarizes the qubit states Alice sends, based on the bit of Alice's `alice_bits` the corresponding basis of `alice_bases`:

| Bit in Alice's `alice_bits` | Corresponding basis in `alice_bases` | Qubit state sent |
|:----------------:|:-----------------:|:-----------------:|
| 0 | Z |$$|0\rangle$$ |
| 0 | X |$$|+\rangle$$ |
| 1 | Z |$$|1\rangle$$ |
| 1 | X |$$|-\rangle$$ |

Write a function to encode `alice_bits` using the corresponding basis in `alice_bases`. The function should return a list of quantum circuits called `encoded_qubits`, with each circuit in the list containing the appropriate gates to prepare the correct qubit state for the corresponding bit and basis. Do not add measurements to the circuits yet - Bob will add measurements to these qubits after receiving them.

In [10]:
# Block 4 - Write a function to encode qubits using alice_bits and alice_bases

def encode(alice_bits, alice_bases):
    encoded_qubits = []
    for i in range(len(alice_bits)):
        # create a brand new quantum circuit called qc. Remember that the qubit will be in state |0> by default
        qc = QuantumCircuit(1,1)

        if alice_bases[i] == "Z":
            # We are encoding in the z basis
            if alice_bits[i] == "0":
                # We want to encode a |0> state, as states are intialized
                # in |0> by default we don't need to add anything here
                pass 
            
            elif alice_bits[i] == "1":
                # We want to encode a |1> state
                # We apply an X gate to generate |1>
                qc.x(0) 
                
        elif alice_bases[i] == "X":
            # 1 Means we are encoding in the x basis
            if alice_bits[i] == "0":
                # We apply an H gate to generate |+>
                qc.h(0)
            elif alice_bits[i] == "1":
                # We apply an X and an H gate to generate |->
                qc.x(0)
                qc.h(0)
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
    return encoded_qubits

## Create Alice's encoded_qubits

Let's use the function you defined above to create Alice's qubits!

In [23]:
# Block 5 - Call your function from the previous block to create Alice's list encoded_qubits

encoded_qubits = encode(alice_bits, alice_bases)

<a id="step4"></a>
## Step 4: Alice sends qubits to Bob

Alice now sends the encoded qubits to Bob. We cannot implement this step in this simulation. In a real application, Alice might send her qubits over an optical fibre, which would be the quantum channel.

<a id="step2"></a>
## Step 5: Bob randomly selectes bases

Bob has received Alice's qubits, and now Bob needs to randomly select bases in which to measure each qubit that Alice sent him.

Write a function that takes in the number of bases that Bob needs to pick and returns a list called `bob_bases` of randomly selected measurement bases (either "X" or "Z").

In [12]:
# Block 6 - Write a function to pick Bob's bases

def generate_bob_bases(length):
    # Similar to before we store the bases that Bob will measure in a list
    bob_bases = []
    
    for i in range(length):
        # Again we use getrandbits to generate a 0 or 1 randomly
        basis = getrandbits(1)
        if basis == 0:
            bob_bases.append("Z")
        else:
            bob_bases.append("X")
        
    # return the list of random bases to measure in
    return bob_bases

## Create Bob's list of bases

Create Bob's list of bases `bob_bases` by calling the function you defined in the previous block. 

We can see what it looks like by printing the first 10 elements, it should look like a list of random 'X's or 'Z's 

In [13]:
# Block 7 - Use your function create bob_bases
bob_bases = generate_bob_bases(N)

# Preview the first 10 elements of each:
print("bob's first 10 bases: ", bob_bases[:10])

bob's first 10 bases:  ['X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z']


<a id="step5"></a>
## Steps 6 and 7: Bob measures Alice's qubits using `bob_bases` and decodes them to bits

Bob now has to measure the qubits Bob received from Alice using the list of random bases `bob_bases`. Write a function that measures each circuit in `encoded_qubits` in the corresponding basis in `bob_bases`. 

You will need to correctly add measurements to each of the circuits in `encoded_qubits`, depending on whether the measurement is in the Z basis, or in the X basis. Simulate each circuit using the QASM simulator, with `counts = 1`. Extract the counts from the results of the simulation, and then use the code `measured_bit = max(counts, key=counts.get)` to extract the measured bit (0 or 1) from the counts. Create a list called `bob_bits` that contains the list of bits generate by each measurement. This is Bob's decoded list of bits!

In [14]:
# Block 8 - Write a function to make measurements on `encoded_qubits` using `bob_bases`, and decode the results of the measurements to bits.
def measure(bob_bases, encoded_qubits):

    
    # Stores the results of Bob's measurements
    bob_bits = []
    backend = Aer.get_backend('qasm_simulator')
    for i in range(len(encoded_qubits)):
        qc = encoded_qubits[i]
        
        if bob_bases[i] == "Z":
            # We want to measure in Z basis
            qc.measure(0,0)

        elif bob_bases[i] == "X":
            # We want to measure in X basis
            qc.h(0)
            qc.measure(0,0)
        
        # Now that the measurements have been added to the circuit, let's run them.
        job = execute(qc, backend=backend, shots = 1) # increase shots if running on hardware
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bob_bits.append(measured_bit) 
        
    return bob_bits

## Measure the qubits that Bob recieved from Alice

Let's call the function you wrote in the previous block to perform measurements on the bits Alice sent to Bob, and generate `bob_bits`

In [15]:
# Block 9 - Call your function from the previous block to generate Bob's list of measured bits, bob_bits

bob_bits = measure(bob_bases, encoded_qubits)

<a id="step6"></a>
## Steps 8 and 9: Alice and Bob compare their bases and bits, and generate their key

In this step, Alice and Bob will compare their list of bases, and throw away the bits for which the corresponding bases do not match. The remaining bits, for which their bases match, are guaranteed to match, and can be used as the key! Finally, to check if there was an eavesdropper, Alice and Bob compare the first few bits of their key.

In the block below, write a function that will generate a list called `agreeing_indices` which contains a list of indices at which Alice and Bob's bases (`alice_bases` and `bob_bases`) match.

In [16]:
# Block 10 - Write a function to generate a list agreeing_indices containing the list of indices where alice_bases and bob_bases match
def find_matching_bases(alice_bases,bob_bases):
    agreeing_indices = []
    for i in range(len(alice_bases)):
        if alice_bases[i]==bob_bases[i]:
            agreeing_indices.append(i)
            
    return agreeing_indices

## Finding matching bases

Let's use the function you created in the previous block to generate the list `agreeing_indices` from `alice_bases` and `bob_bases` that match.

As a check of your implementations so far, try printing the length of `agreeing_indices`. What do you expect this length to be (approximately)? Remember that both Alice and Bob chose randomly from "Z" and "X" to generate their lists of bases.

In [24]:
# Block 11 - Call the function you created in the previous block to generate agreeing_indices

agreeing_indices = find_matching_bases(alice_bases, bob_bases)

print(len(agreeing_indices))

247


<a id="step7"></a>

Now that Alice and Bob know which of their bases match, they can use the list `agreeing_indices` to generate their keys!

Write a function that uses `agreeing_indices` and a list of bits to generate a key that only contains the bits at the indices given by `agreeing_indices`, and then use that function to generate both Alice and Bob's keys.

In [18]:
# Block 12 - Write a function that uses agreeing_indices and a list of bits to generate a key

def generate_key(agreeing_indices, bits):
    key = []
    for index in agreeing_indices:
        key.append(bits[index])
    return key

In [19]:
# Block 13 - Use your function from the previous block to construct Alice and Bob's keys

alice_key = generate_key(agreeing_indices, alice_bits)
bob_key = generate_key(agreeing_indices, bob_bits)

Finally, Alice and Bob compare the first few bits of their key to check if there was an eavesdropper. Since we have not implemented an eavesdropper, we expect all bits of their keys to be the same - check this in the next block! If you get a mismatch, you may need to revisit and correct your code for some of the previous steps.

In [20]:
# Block 14 - Check if Alice and Bob's keys are the same.

# Preview the first 10 elements of each Key:
print("alice_key: ", alice_key[:10])
print("bob_key: ", bob_key[:10])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)

alice_key:  ['1', '1', '0', '1', '1', '0', '1', '0', '0', '0']
bob_key:  ['1', '1', '0', '1', '1', '0', '1', '0', '0', '0']
Alice's key is equal to Bob's key:  True


# Congratulations! You have implemented the BB84 QKD protocol successfully!

## Encryping and decrypting messages using the key

Once the key is generated, Alice and Bob can use it to send and receive messages! In the block below, we have created two helper functions `encrypt_message` and `decrypt_message` to encrypt and decrypt the message. `encrypt_message` takes the unencrypted message along with the key to encode the message, while `decrypt_message` takes the encrypted message and the key to decrypt the message.

In [21]:
# BLOCK 15 - creating helper functions to encrypt and decrypt messages using QKD

import binascii

def encrypt_message(unencrypted_string, key):
    # Convert ascii string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # created the encrypted string using the key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # created the unencrypted string using the key
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Convert bitstring into
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

In the block below, use these two functions to encrypt and decrypt the message using the key you had generated earlier!

In [22]:
# BLOCK 16 - sending and receiving messages using QKD

message = "QKD is cool!"
print("Original Messge:", message)
encrypted_message = encrypt_message(message, alice_key)
print("Encrypted message:", encrypted_message)
decrypted_message = decrypt_message(encrypted_message, bob_key)
print("Decrypted message:", decrypted_message)

Original Messge: QKD is cool!
Encrypted message: 100010110110101111001110101100010010001100100000001111000111010111111111110010001011000001100011
Decrypted message: QKD is cool!


## Optional extensions

We did not implement an eavesdropper in this protocol so far. The eavesdropper would intercept Alice's qubits, make measurements on them, and then pass on the qubits (after they have been measured) to Bob to try to evade detection. Can you implement an eavesdropper function and show that in the presence of an eavesdropper, Alice and Bob's keys will not match? Therefore, Alice and Bob can detect the eavesdropper's presence by comparing the first few bits of their keys and checking for any differences!

Suppose Alice and Bob check the first 20 bits of their keys. What is the probability of successful eavesdropping?

Can you think of any other eavesdropping strategies that would be more successful?